Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [4]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [5]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/mouse-biosamples/altius-biosamples.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [6]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [7]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,mouse_id:skip,model_organism_sex,starting_amount:integer,starting_amount_units,date_obtained:date,model_organism_age,model_organism_age_units,organism,donor,source,lab,award
0,080b20c5-cff7-4e8b-8f46-97affd5c40f6,ENCBS110XTZ,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,B6Cast F1 PND4 3x Males 05 adrenal gland for A...,5,male,3.0,items,2020-01-23,4.0,day,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
1,ed68f3e1-608c-4fc2-aa18-77fc9c188bb7,ENCBS209QZR,NaN,/biosample-types/tissue_UBERON_0002113/,kidney,B6Cast F1 PND4 3x Males 05 kidney for Altius S...,5,male,3.0,items,2020-01-23,4.0,day,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
2,38c2d6c1-25a1-4212-8c39-0f6cdb758876,ENCBS561TMO,NaN,/biosample-types/tissue_UBERON_0002037/,cerebellum,B6Cast F1 PND4 3x Males 05 cerebellum for Alti...,5,male,3.0,items,2020-01-23,4.0,day,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
3,9e6f7260-7211-44ca-9896-177f4d0e979f,ENCBS958KUJ,NaN,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,B6Cast F1 PND4 3x Males 05 gastrocnemius for A...,5,male,3.0,items,2020-01-23,4.0,day,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
4,cce20696-7d4a-4dbb-9cee-d45397e1e105,ENCBS777ZHB,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,B6Cast F1 PND4 3x Males 06 adrenal gland for A...,6,male,3.0,items,2020-01-23,4.0,day,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,127d251b-5700-4689-a76c-45714a810258,ENCBS703MDQ,NaN,/biosample-types/tissue_UBERON_0002369/,adrenal gland,B6Cast F1 PND 18-20 months Female 04 adrenal g...,4,female,1.0,items,NaT,NaN,NaN,/organisms/mouse,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
571,c575dafd-0cd9-41ca-998c-d69449085733,ENCBS752ONG,NaN,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,B6Cast F1 PND 18-20 months Male 03 gastrocnemius,3,male,1.0,items,NaT,NaN,NaN,/organisms/mouse,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
572,eafeb508-a5a4-4470-9b85-5e409cf6d793,ENCBS861KNF,NaN,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,B6Cast F1 PND 18-20 months Male 04 gastrocnemius,4,male,1.0,items,NaT,NaN,NaN,/organisms/mouse,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
573,901b789d-5010-48c9-8151-84a63bacae06,ENCBS216EDC,NaN,/biosample-types/tissue_UBERON_0001388/,gastrocnemius,B6Cast F1 PND 18-20 months Female 03 gastrocne...,3,female,1.0,items,NaT,NaN,NaN,/organisms/mouse,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443


In [8]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Biosamples

In [9]:
#biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

#for i, row in biosample.iterrows():
#    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
#        obj = server.get_json(row.accession)
#        biosample_ontology = obj['biosample_ontology']
#        if isinstance(biosample_ontology, dict):
#            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
#            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
#        biosample.loc[i, 'source'] = obj['source']['@id']
#            
#biosample

In [10]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [11]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

## Forgot to post model_organism_sex

In [19]:
print(spreadsheet_name)
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
updates = []
for i, row in biosamples.iterrows():
    biosample = server.get_json(row['accession'])
    model_organism_age = biosample.get('model_organism_sex')
    if model_organism_age != row['model_organism_sex']:
        print("{} is {} should be {}".format(row['accession'], model_organism_age, row['model_organism_sex']))
        if biosample['status'] == "in progress":
            updates.append(server.patch_json(
                '/biosamples/{}/'.format(row['accession']),
                {'model_organism_sex': row['model_organism_sex']}))
        else:
            print('Unable to update {} status {}'.format(row['accession'], biosample['status']))

/home/diane/woldlab/ENCODE/mouse-biosamples/altius-biosamples.xlsx
ENCBS886BLQ is None should be female
Unable to update ENCBS886BLQ status released
ENCBS727HZM is None should be female
ENCBS644ZIZ is None should be female
ENCBS559PTX is None should be female
ENCBS118MVG is None should be female
Unable to update ENCBS118MVG status released
ENCBS120VNJ is None should be female
ENCBS846CFE is None should be female
ENCBS519KGT is None should be female
ENCBS588OTO is None should be male
ENCBS799RYT is None should be male
ENCBS591RPK is None should be male
ENCBS766LFK is None should be male
ENCBS368WXW is None should be male
ENCBS730GFD is None should be male
ENCBS357DOL is None should be male
ENCBS078GAU is None should be male
ENCBS824HLN is None should be female
ENCBS725CYF is None should be female
ENCBS179RNJ is None should be female
ENCBS927BKO is None should be female
ENCBS622FYD is None should be female
ENCBS275VAC is None should be female
ENCBS973QQI is None should be female
ENCBS900

ENCBS702OJF is None should be male
ENCBS378WXL is None should be male
ENCBS816FLU is None should be male
ENCBS746PCF is None should be male
ENCBS657CGA is None should be female
ENCBS527XPI is None should be female
ENCBS994LPC is None should be female
ENCBS185GMZ is None should be female
ENCBS530AXR is None should be female
ENCBS170UDX is None should be female
ENCBS710SAL is None should be female
ENCBS709PIP is None should be female
ENCBS607KWL is None should be female
ENCBS888DCZ is None should be female
ENCBS737BPG is None should be female
ENCBS699PEY is None should be female
ENCBS693DPN is None should be female
ENCBS578XBR is None should be female
ENCBS775KKZ is None should be female
ENCBS682WFB is None should be female
ENCBS204IKD is None should be male
ENCBS350BQS is None should be male
ENCBS161MWT is None should be male
ENCBS398KZG is None should be male
ENCBS154PWG is None should be male
ENCBS105VUY is None should be male
ENCBS502YBW is None should be male
ENCBS248TGZ is None sho

In [17]:
biosample['status']

'released'